In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
names = pd.read_csv('./stockdata/stockNames.csv', sep=',', header=None)
names.head()

,0,1,2,3,4,5,6,7,8,9,...,480,481,482,483,484,485,486,487,488,489
0,AA,AAPL,ABC,ABI,ABK,ABS,ABT,ACE,ACS,A,...,XEL,XL,XLNX,XOM,XRX,XTO,YHOO,YUM,ZION,ZMH


In [3]:
stockdata = pd.read_csv('./stockdata/stockData.csv', sep=',', header=None)
stockdata = stockdata.values
stockdata.shape

(490, 1000)

## part a

Pick the "best" single stock: max possible earnings over the course of 1000 trading days

In [4]:
percent_return = stockdata[:, -1] / stockdata[:, 0]

In [5]:
best_stock = np.argmax(percent_return)
names[best_stock]

0    COH
Name: 112, dtype: object

In [6]:
stockdata[best_stock, 0], stockdata[best_stock, -1]

(4.14, 34.43)

In [7]:
percent_return[best_stock]

8.316425120772948

Equal amount of money placed in each stock

In [8]:
np.sum(1 / len(percent_return) * percent_return)

1.5802565023247446

CRP strategy that allocates an equal percentage of the portfolio to each stock

In [9]:
percent_change = []
for t in range(1, len(stockdata.T)):
    percent_change.append(stockdata[:, t] / stockdata[:, t - 1])
percent_change = np.vstack(percent_change)
percent_change.shape

(999, 490)

In [10]:
def compute_earnings(alloc):
    """
    Outputs the earnings over all stock data given
    a fixed allocation of funds (i.e. CRP rebalancing
    based on the change in earnings to maintain the
    allocation)
    """
    final_earnings = 1
    for change in percent_change:
        earned = np.dot(change, alloc)
        final_earnings *= earned
    return final_earnings

In [11]:
compute_earnings([1 / len(stockdata)] * len(stockdata))

1.6327365103499576

## part b

In [12]:
log_pc_change = np.zeros((len(stockdata.T) - 1, len(stockdata)))
for t in range(1, len(stockdata.T)):
    log_pc_change[t - 1] = np.log(stockdata[:, t] / stockdata[:, t - 1])

In [13]:
def simplex_proj(y):
    """
    implementation based on 
    https://eng.ucmerced.edu/people/wwang5/papers/SimplexProj.pdf
    """
    n_features = y.shape[0]
    u = np.sort(y)[::-1]
    cssv = np.cumsum(u) - 1
    ind = np.arange(n_features) + 1
    cond = u - cssv / ind > 0
    theta = cssv[cond][-1] / float(ind[cond][-1])
    return np.maximum(y - theta, 0)

In [14]:
def vectorized_gradient(single_alloc, data):
    return np.sum(
        np.divide(data.T,
                  np.matmul(data, single_alloc.T)), axis=1)

In [15]:
extreme_alloc = np.zeros(490)
extreme_alloc[0] = 1
np.linalg.norm(simplex_proj(extreme_alloc))

1.0

In [16]:
data = log_pc_change + 1
_, N_stocks = data.shape

D = np.sqrt(2)  # the two "extremes" that define D are 100% in 1 stock <-> 100% in another stock, 2-norm == sqrt(2)
G = 1  # upper bound on the l2-norm of the gradient for any x in the simplex projection
T = int((4 * D**2 * G**2) / (0.005**2))  # epsilon=0.005 just so this runs in a reasonable time
grad = np.zeros((N_stocks, T)) 
alloc = np.zeros((N_stocks, T))
alloc[:, 0] = 1 / N_stocks  # start with equal percentage in each stock
eta = D / (G * np.sqrt(T))
print("T = {0}, eta = {1}".format(T, eta))
for i in range(1, T):
    grad[:,i] = vectorized_gradient(alloc[:,i-1], data)
    upd = alloc[:,i-1] + eta*grad[:,i]  
    alloc[:,i] = simplex_proj(upd)
z = np.sum(alloc, axis=1) / T

T = 320000, eta = 0.0025


In [17]:
np.sum(z)

0.9999999999999987

In [28]:
allocation = ','.join(['{0:.4f}'.format(a) for a in z])
with open('./hw3_p1b_allocations.txt', 'w+') as fh:
    fh.write('{0}\n'.format(allocation))

In [18]:
names[np.argmax(z)]

0    COH
Name: 112, dtype: object

In [19]:
compute_earnings(z)

8.612060485915375

## part c

In [20]:
def gradient(grad, alloc, data):
    return grad + data / np.matmul(alloc, data)

In [21]:
def OGD(data, eta):
    T, N = data.shape
    data = data + 1
    alloc = np.zeros((N, T))
    alloc[:, 0] = 1 / N
    grad = np.zeros((N, T)) 
    for i in range(1, T):
        grad[:,i] = gradient(grad[:,i-1], alloc[:,i-1], data[i-1])
        upd = alloc[:,i-1] + eta*grad[:,i]
        alloc[:,i] = simplex_proj(upd)
    return {'portfolio': alloc, 'gradient': grad}

Trying different learning rates

In [22]:
portfolio = OGD(log_pc_change, 0.0005)
compute_earnings(portfolio['portfolio'][:, -1])

6.912924959928768

In [23]:
portfolio = OGD(log_pc_change, 0.001)
compute_earnings(portfolio['portfolio'][:, -1])

8.093667713844395

In [24]:
portfolio = OGD(log_pc_change, 0.01)
compute_earnings(portfolio['portfolio'][:, -1])

8.33487486827706

In [25]:
portfolio = OGD(log_pc_change, 0.1) # **best return!
compute_earnings(portfolio['portfolio'][:, -1])

8.429289751492883

In [26]:
portfolio = OGD(log_pc_change, 0.15)
compute_earnings(portfolio['portfolio'][:, -1])

8.169103539446795